#Week-4 RetailSales Dashboard

In [1]:
from google.colab import files
uploaded = files.upload()


Saving cleaned_sales_data.csv to cleaned_sales_data.csv


In [2]:
import pandas as pd
df_check = pd.read_csv("cleaned_sales_data.csv")
print(df_check.head())


   sale_id   sale_date  quantity product_name     category    price     cost  \
0        1  2025-07-01         3       Laptop  Electronics  54000.0  48000.0   
1        2  2025-07-01        10        Mouse  Electronics    700.0    500.0   
2        4  2025-07-02        15   Headphones  Electronics   1500.0   1000.0   
3        5  2025-07-02        20        Shirt     Clothing   1200.0    800.0   
4        6  2025-07-03        10        Shoes     Clothing   2500.0   1800.0   

   store_name region employee_name             role   revenue   profit  \
0  All things  South   Rohit Kumar  Sales Associate  162000.0  18000.0   
1  Delhi Mall  North       Ankitha          Cashier    7000.0   2000.0   
2  All things  South   Rohit Kumar  Sales Associate   22500.0   7500.0   
3  Delhi Mall  North       Ankitha          Cashier   24000.0   8000.0   
4  All things  South   Rohit Kumar  Sales Associate   25000.0   7000.0   

   profit_margin  
0      11.111111  
1      28.571429  
2      33.333333 

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, round
spark = SparkSession.builder.appName("RetailSalesETL").getOrCreate()
df = spark.read.csv("cleaned_sales_data.csv", header=True, inferSchema=True)
df.show()



+-------+----------+--------+------------+-----------+-------+-------+----------+------+-------------+---------------+--------+-------+------------------+
|sale_id| sale_date|quantity|product_name|   category|  price|   cost|store_name|region|employee_name|           role| revenue| profit|     profit_margin|
+-------+----------+--------+------------+-----------+-------+-------+----------+------+-------------+---------------+--------+-------+------------------+
|      1|2025-07-01|       3|      Laptop|Electronics|54000.0|48000.0|All things| South|  Rohit Kumar|Sales Associate|162000.0|18000.0| 11.11111111111111|
|      2|2025-07-01|      10|       Mouse|Electronics|  700.0|  500.0|Delhi Mall| North|      Ankitha|        Cashier|  7000.0| 2000.0| 28.57142857142857|
|      4|2025-07-02|      15|  Headphones|Electronics| 1500.0| 1000.0|All things| South|  Rohit Kumar|Sales Associate| 22500.0| 7500.0| 33.33333333333333|
|      5|2025-07-02|      20|       Shirt|   Clothing| 1200.0|  800.0|

In [4]:
# 3. Transformation: Revenue, Profit, Profit Margin by Category
category_metrics = (
    df.groupBy("category")
      .agg(
          _sum(col("revenue")).alias("total_revenue"),
          _sum(col("profit")).alias("total_profit")
      )
      .withColumn("profit_margin", round((col("total_profit")/col("total_revenue"))*100, 2))
)

print("Category-level Metrics:")
category_metrics.show()

# 4. Save results as CSV (simulating Delta/CSV output in Databricks)
category_metrics.toPandas().to_csv("category_metrics.csv", index=False)

# 5. Register TempView for SQL queries
df.createOrReplaceTempView("sales_data")

# 6. Run SQL query: Top 3 best-selling products
top_products = spark.sql("""
    SELECT product_name, SUM(quantity) AS total_sold
    FROM sales_data
    GROUP BY product_name
    ORDER BY total_sold DESC
    LIMIT 3
""")

print("Top 3 Best-Selling Products:")
top_products.show()

# Save SQL output
top_products.toPandas().to_csv("top_3_products.csv", index=False)


Category-level Metrics:
+-----------+-------------+------------+-------------+
|   category|total_revenue|total_profit|profit_margin|
+-----------+-------------+------------+-------------+
|Electronics|     191500.0|     27500.0|        14.36|
|   Clothing|      49000.0|     15000.0|        30.61|
|  Furniture|      60000.0|     15000.0|         25.0|
+-----------+-------------+------------+-------------+

Top 3 Best-Selling Products:
+------------+----------+
|product_name|total_sold|
+------------+----------+
|       Shirt|        20|
|  Headphones|        15|
|       Shoes|        10|
+------------+----------+



In [6]:
category_metrics.toPandas().to_csv("category_metrics.csv", index=False)

# Download result to your laptop
files.download("category_metrics.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>